In [1]:
using MixedLayerModel
using FileIO

using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger());

┌ Info: Precompiling MixedLayerModel [45e51734-b38e-40e9-bdad-5745a9e5fe6d]
└ @ Base loading.jl:1278


In [2]:
# set-up basic 400 ppm simulation
par = basic_params();
par.etype = Sally();
par.a = 1.1;
path = "./output/Sally1.1/";

# run to steady-state
u0, sol = run_mlm(par);

# get output
code = sol.retcode;
println(code);
uf = sol.u;
du = zeros(4);
mlm(du, uf, par, 0.0);
zi,hM,qM,SST = uf;
zb = calc_LCL(zi,hM,qM);
println(uf);
println(du ./ uf);

# save to file
output = Dict("code" => code, "p"=>par, "u0" => u0, "uf" => uf, "du/u" => du./uf, 
"we" => we(uf,par,par.etype), "zb" => zb, "zc" => zi-zb,
"RHsurf" => RH(0.0, hM, qM), "LHF" => calc_LHF(uf,par), "SHF" => calc_SHF(uf,par),
"ΔR" => calc_cloudtop_RAD(uf,par,par.rtype), "OHU" => calc_OHU(uf,par,par.stype));
save(path*"co2_400.jld2", output)

euler, dt=4 hrs
 19.683092 seconds (80.24 M allocations: 3.494 GiB, 4.27% gc time)
Success
[1150.6332556144173, 312606.84575708106, 0.007982343882468011, 290.0]
[2.2132663819572702e-10, 3.421317211891941e-10, 8.097017519369867e-10, 0.0]


In [5]:
##########
# increase CO2 but fix SST
##########

# set co2
newCO2 = 600.0

# load initial condition from file
path = "./output/Sally1.1/";
output = load(path*"co2_400.jld2");
u0 = output["uf"];
OHU = output["OHU"];
println(u0);
mlm(du, u0, par, 0.0);
println(du ./ u0);

# increase CO2, let SST evolve and check cloud changes
par.CO2 = newCO2;
u0, sol = run_mlm_from_init(u0, par);

# get output
code = sol.retcode;
println(code);
uf = sol.u;
du = zeros(4);
mlm(du, uf, par, 0.0);
zi,hM,qM,SST = uf;
zb = calc_LCL(zi,hM,qM);
println(uf);

# save to file
output = Dict("code" => code, "p" => par, "u0" => u0, "uf" => uf, "du/u" => du./uf, 
"we" => we(uf,par,par.etype), "zb" => zb, "zc" => zi-zb,
"RHsurf" => RH(0.0, hM, qM), "LHF" => calc_LHF(uf,par), "SHF" => calc_SHF(uf,par),
"ΔR" => calc_cloudtop_RAD(uf,par,par.rtype), "OHU" => calc_OHU(uf,par,par.stype))
save(path*"co2_upstep_fixSST_"*string(Int(newCO2))*".jld2", output)

[1150.6332556144173, 312606.84575708106, 0.007982343882468011, 290.0]
[2.2132663819572702e-10, 3.421317211891941e-10, 8.097017519369867e-10, 0.0]
euler
dt=240.0 mins


ODE   0%|                                               |  ETA: N/A
ODE  17%|███████▉                                       |  ETA: 0:00:19
ODE 100%|███████████████████████████████████████████████| Time: 0:00:06


  8.391408 seconds (53.25 M allocations: 1.825 GiB, 4.24% gc time)
Success
[1005.6038148361873, 313602.28524279705, 0.008463360090647642, 290.0]


In [8]:
##########
# increase CO2 and let SST evolve
##########

# load initial condition from file
path = "./output/Sally1.1/";
output = load(path*"co2_400.jld2");
u0 = output["uf"];
OHU = output["OHU"];
println(u0);
mlm(du, u0, par, 0.0);
println(du ./ u0);

# set OHU, let SST evolve and check cloud changes
newCO2 = 410.0;
par.CO2 = newCO2;
par.Hw = 1;
par.OHU = OHU;
par.stype = varSST();
u0, sol = run_mlm_from_init(u0, par);

# get output
code = sol.retcode;
println(code);
uf = sol.u;
du = zeros(4);
mlm(du, uf, par, 0.0);
zi,hM,qM,SST = uf;
zb = calc_LCL(zi,hM,qM);
println(uf);
println(du ./ uf);

# # save to file
# output = Dict("code" => code, "p" => par, "u0" => u0, "uf" => uf, "du/u" => du./uf, 
# "we" => we(uf,par,par.etype), "zb" => zb, "zc" => zi-zb,
# "RHsurf" => RH(0.0, hM, qM), "LHF" => calc_LHF(uf,par), "SHF" => calc_SHF(uf,par),
# "ΔR" => calc_cloudtop_RAD(uf,par,par.rtype), "OHU" => calc_OHU(uf,par,par.stype))
# save(path*"co2_upstep_"*string(Int(newCO2))*".jld2", output)

[1150.6332556144173, 312606.84575708106, 0.007982343882468011, 290.0]
[-8.35408566864429e-8, 3.04077820188782e-9, 5.0398934906078256e-8, 0.0]
euler
dt=10.0 mins


ODE   0%|                                               |  ETA: N/A
ODE   0%|                                               |  ETA: N/A
ODE   0%|▏                                              |  ETA: 0:20:26
ODE   0%|                                               |  ETA: N/A
ODE   1%|▍                                              |  ETA: 0:20:09
ODE   0%|                                               |  ETA: N/A
ODE   1%|▌                                              |  ETA: 0:20:08
ODE   0%|                                               |  ETA: N/A
ODE   1%|▋                                              |  ETA: 0:20:22
ODE   0%|                                               |  ETA: N/A
ODE   2%|▉                                              |  ETA: 0:20:25
ODE   0%|                                               |  ETA: N/A
ODE   2%|█                                              |  ETA: 0:20:27
ODE   0%|                                               |  ETA: N/A
ODE   2%|█▏             

LoadError: InterruptException:

In [ ]:
##########
# increase CO2 and let SST evolve
##########

# load initial condition from file
path = "./output/Sally1.1/";
output = load(path*"co2_400.jld2");
u0 = output["uf"];
OHU = output["OHU"];
println(u0)

# set OHU, let SST evolve and check cloud changes
newCO2 = 600.0;
par.CO2 = newCO2;
par.Hw = 1;
par.OHU = OHU;
par.stype = varSST();
u0, sol = run_mlm_from_init(u0, par);

# get output
code = sol.retcode;
println(code);
uf = sol.u;
du = zeros(4);
mlm(du, uf, par, 0.0);
zi,hM,qM,SST = uf;
zb = calc_LCL(zi,hM,qM);
println(uf);

# save to file
output = Dict("code" => code, "p" => par, "u0" => u0, "uf" => uf, "du/u" => du./uf, 
"we" => we(uf,par,par.etype), "zb" => zb, "zc" => zi-zb,
"RHsurf" => RH(0.0, hM, qM), "LHF" => calc_LHF(uf,par), "SHF" => calc_SHF(uf,par),
"ΔR" => calc_cloudtop_RAD(uf,par,par.rtype), "OHU" => calc_OHU(uf,par,par.stype))
save(path*"co2_upstep_"*string(Int(newCO2))*".jld2", output)